# Problemas de Satisfacción de Restricciones
- Ricardo Méndez 21289
- Sara Echeverría 21371
- Francisco Castillo 21562
- Repositorio de Github: https://github.com/bl33h/constraintSatisfactionProblems

# Task 1 - Teoría

## Algoritmo AC-3 y su relación con el algoritmo de backtracking search
El algoritmo AC-3 optimiza la resolución de problemas de satisfacción de restricciones (CSP) eliminando previamente valores que no pueden satisfacer las restricciones entre variables, reduciendo así el espacio de búsqueda. Por su parte, el algoritmo de backtracking explora este espacio simplificado para encontrar soluciones, retrocediendo cuando una ruta no conduce a una solución válida. Su relación radica en la combinación de estos, pues dan como resultado una estrategia eficaz en la cual AC-3 reduce el espacio de posibilidades y el algoritmo de backtracking navega por este espacio optimizado en busca de la solución (Gao, 2024).


## Arc Consistency
El término de ‘Arc Consistency’ describe un método eficaz para abordar problemas de satisfacción de restricciones, mediante el cual se eliminan las opciones inviables que no se ajustan a las restricciones establecidas (Brown, 2010).

## Referencias
- Brown. (2010). CSPs: Arc Consistency. The University of British Columbia. https://www.cs.ubc.ca/~kevinlb/teaching/cs322%20-%202009-10/Lectures/CSP3.pdf
- Gao. (2024). Constraint Satisfaction Problems: Backtracking Search and Arc Consistency. University of Waterloo. https://cs.uwaterloo.ca/~a23gao/cs486686_s19/slides/lec05_csp_arc_consistency_backtracking_search_nosol.pdf

# Task 2

## Restricciones
- 4 estudiantes y 7 exámenes diferentes
- Todos los exámenes se realizan en tres días
- Ningún estudiante puede tener más de un exámen por día.
- Los estudiantes que toman el mismo curso no pueden tener exámenes programados para el mismo día

In [614]:
import time
import random

random.seed(123)

In [615]:
class MeasureTime:
    def __init__(self, ):
        self.start = 0
        self.end = 0

    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time.time()
        print(f'Time elapsed: {self.end - self.start} seconds')
        
    def get_time(self):
        return self.end - self.start

In [616]:
measure_time = MeasureTime()

In [617]:
times = {
    'backtracking': None,
    'beam_search': None,
    'local_search': None
}

## Definición de Valores

In [618]:
exams = ['Math', 'Physics', 'Chemistry', 'History', 'Biology', 'Geography', 'Literature']
days = ['Monday', 'Tuesday', 'Wednesday']
students_names = ['Francisco', 'Sara', 'Ricardo', 'Fulanito']

In [619]:
class Student:
    def __init__(self, name):
        self.name = name
        self.exams = {'None'}

    def add_exam(self, exam):
        self.exams.add(exam)

    def __str__(self):
        return self.name

    def __repr__(self):
        return self.name

    def __eq__(self, other):
        return self.name == other.name

    def __hash__(self):
        return hash(self.name)

### Creación de estudiantes y asignación de exámenes (para cumplir con las restricciones)

In [620]:
# Create Students
students = []
for name in students_names:
    students.append(Student(name))

# Assign Exams to Students
for i in range(len(exams)):
    students[i % len(students)].add_exam(exams[i])

# Assign Exams until each student has 3 different exams
for student in students:
    while len(student.exams) < 4:
        additional_exam = random.choice(exams)
        student.add_exam(additional_exam)

# Print students and their exams
print('Students and their exams:')
for student in students:
    print(f'{student}: {[exam for exam in student.exams]}')

Students and their exams:
Francisco: ['None', 'Chemistry', 'Math', 'Biology']
Sara: ['None', 'Geography', 'Physics', 'Math']
Ricardo: ['None', 'Chemistry', 'History', 'Literature']
Fulanito: ['None', 'Chemistry', 'History', 'Math']


### Definición de Variables, Dominios y Restricciones

In [621]:
variables = []
for student in students:
    for day in days:
        variables.append((student, day))

domains = {}
for var in variables:
    domains[var] = [exam for exam in var[0].exams]
    
print('Domains:')
domains

Domains:


{(Francisco, 'Monday'): ['None', 'Chemistry', 'Math', 'Biology'],
 (Francisco, 'Tuesday'): ['None', 'Chemistry', 'Math', 'Biology'],
 (Francisco, 'Wednesday'): ['None', 'Chemistry', 'Math', 'Biology'],
 (Sara, 'Monday'): ['None', 'Geography', 'Physics', 'Math'],
 (Sara, 'Tuesday'): ['None', 'Geography', 'Physics', 'Math'],
 (Sara, 'Wednesday'): ['None', 'Geography', 'Physics', 'Math'],
 (Ricardo, 'Monday'): ['None', 'Chemistry', 'History', 'Literature'],
 (Ricardo, 'Tuesday'): ['None', 'Chemistry', 'History', 'Literature'],
 (Ricardo, 'Wednesday'): ['None', 'Chemistry', 'History', 'Literature'],
 (Fulanito, 'Monday'): ['None', 'Chemistry', 'History', 'Math'],
 (Fulanito, 'Tuesday'): ['None', 'Chemistry', 'History', 'Math'],
 (Fulanito, 'Wednesday'): ['None', 'Chemistry', 'History', 'Math']}

In [622]:
# Constraints
# NOTE THAT, BY THE NATURE OF THE ASSIGNMENT, THE STUDENT CAN'T TAKE MORE THAN ONE EXAM PER DAY
def constraint_all_exams_taken(assignment: dict) -> bool:
    # All values of the exams list must be in the values of the dict
    exams_taken = []
    for var in assignment:
        if assignment[var] != 'None':
            exams_taken.append(assignment[var])

    return set(exams) == set(exams_taken)  # If all the exams in the list are part of the assignment, return True

In [623]:
def constraint_one_exam_type_per_day(assignment: dict) -> bool:
    # At max one exam of each type per day
    exams_taken = {}  # day: list[exams]
    for day in days:
        exams_taken[day] = []
    for var in assignment:
        if assignment[var] != 'None':
            if assignment[var] in exams_taken[var[1]]:
                return False
            exams_taken[var[1]].append(assignment[var])
    return True

In [624]:
def print_result(result: dict):
    # Sort by day
    sorted_result = sorted(result.items(), key=lambda x: x[0][1])
    for student_day, exam in sorted_result:
        if exam == 'None':
            continue
        student, day = student_day
        print(f'{day}:\t\t{student} - {exam}')

## Backtracking Search 

In [625]:
def backtracking_search(domains):
    def recursive_backtracking(assignment):
        if len(assignment) == len(domains):
            return assignment

        unassigned = [v for v in domains.keys() if v not in assignment]
        first = unassigned[0]
        for value in domains[first]:
            local_assignment = assignment.copy()
            local_assignment[first] = value
            if constraint_one_exam_type_per_day(local_assignment):
                result = recursive_backtracking(local_assignment)
                if result is not None:
                    # If the result contains all the exams
                    if constraint_all_exams_taken(result):
                        return result
        return None

    return recursive_backtracking({})

In [626]:
result = None
backtracking_times = []
# First run
with measure_time:
    result = backtracking_search(domains)
backtracking_times.append(measure_time.get_time())

# Second run
with measure_time:
    result = backtracking_search(domains)
backtracking_times.append(measure_time.get_time())

# Third run
with measure_time:
    result = backtracking_search(domains)
backtracking_times.append(measure_time.get_time())

times['backtracking'] = backtracking_times

Time elapsed: 4.638906240463257 seconds
Time elapsed: 4.712913751602173 seconds
Time elapsed: 4.687774419784546 seconds


In [627]:
print_result(result)
print(f"Time elapsed: {times['backtracking']} seconds")

Monday:		Fulanito - Chemistry
Tuesday:		Sara - Geography
Tuesday:		Fulanito - History
Wednesday:		Francisco - Biology
Wednesday:		Sara - Physics
Wednesday:		Ricardo - Literature
Wednesday:		Fulanito - Math
Time elapsed: [4.638906240463257, 4.712913751602173, 4.687774419784546] seconds


## Beam Search

## Local Search

# Conclusiones
De cada uno de los algoritmos implementados, tome el tiempo que le toma encontrar una solución, y compare no
solo el tiempo, sino también la solución encontrada de cada uno. 

## Tiempos

In [628]:
times

{'backtracking': [4.638906240463257, 4.712913751602173, 4.687774419784546],
 'beam_search': None,
 'local_search': None}

In [630]:
for time in times:
    if times[time] is not None:
        print(f'Average {time} time: {sum(times[time]) / len(times[time])} seconds')

Average backtracking time: 4.679864803949992 seconds
